In [ ]:
import warnings
import re

from scripts import train, infer, evaluate
from AVDeepfake1Mpp.code.metadata_generator import generate_metadata

%load_ext autoreload
%autoreload 2

# ignore certain warnings
warnings.filterwarnings(
    "ignore", 
    message="The video decoding and encoding capabilities of torchvision are deprecated from version 0.22"
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Goal: Run training for BA-TFD on a small subset of AV-Deepfake1M-PlusPlus to test the setup.

Args:

In [18]:
train_args = {
    "config": 'models/batfd/batfd.toml',
    "data_root": 'AVDeepfake1Mpp/data/',
    "batch_size": 8,
    "num_workers": 11,
    "gpus": 1,
    "precision": 32,
    "num_train": 100,
    "num_val": 100,
    "max_epochs": 2,
    "logger": "tensorboard",
    "resume": None
}

In [19]:
# prepare metadata
generate_metadata(train_args["data_root"])

Existing metadata file AVDeepfake1Mpp/data/train_metadata.json has been removed.
Metadata for train saved to AVDeepfake1Mpp/data/train_metadata.json
Existing metadata file AVDeepfake1Mpp/data/val_metadata.json has been removed.
Metadata for val saved to AVDeepfake1Mpp/data/val_metadata.json
Existing file list AVDeepfake1Mpp/data/testA_files.txt has been removed.
File list for testA saved to AVDeepfake1Mpp/data/testA_files.txt


In [ ]:
#train(train_args)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                   | Type                              | Params | Mode 
-------------------------------------------------------------------------------------
0 | video_encoder          | C3DVideoEncoder                   | 2.3 M  | train
1 | audio_encoder          | CNNAudioEncoder                   | 76.4 K | train
2 | video_frame_classifier | FrameLogisticRegression           | 257    | train
3 | audio_frame_classifier | FrameLogisticRegression           | 257    | train
4 | video_boundary_module  | BoundaryModule                    | 1.5 M  | train
5 | audio_boundary_module  | BoundaryModule                    | 1.5 M  | train
6 | fusion                 | ModalFeatureAttnBoundaryMapFusion | 32.8 K | train
7 | frame_loss             | BCEWithLogitsLoss                 | 0      | train
8 | contrast_loss          | Contra

Model initialized!
DataModule initialized!
Trainer initialized!
Load 2608 data in train.
Load 100 data in val.
Load 1419 data in testA.
Epoch 1: 100%|██████████| 12/12 [01:35<00:00,  0.13it/s, v_num={'20250707-184844'}]

`Trainer.fit` stopped: `max_epochs=2` reached.


Epoch 1: 100%|██████████| 12/12 [01:35<00:00,  0.13it/s, v_num={'20250707-184844'}]


Run inference on validation set, as no labels are provided for the AVDeepfake1M test set:

In [22]:
infer_args = {
    "output_dir": "predictions/",
    "config": train_args["config"],
    "checkpoint": "ckpt/batfd/last.ckpt",  # update with actual checkpoint path
    "data_root": train_args["data_root"],
    "num_workers": train_args["num_workers"],
    "subset": "val",  # use val for 'testing'
    "gpus": train_args["gpus"]
}

In [ ]:
infer(infer_args)

Evaluate the model: